In [30]:
#importing libraries
import numpy as np
import cv2

WIND_X = 1000
WIND_Y = 230

#creating base class
class Figure:
    def __init__(self, x, y, vx, vy, ay = 0):
        self.x  = x
        self.y  = y
        self.vx = vx
        self.vy = vy
        self.ay = ay
    
    def draw(self, img):
        pass
    
    def _move(self, shift = 0):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x <= shift or self.x >= WIND_X - shift):
            self.vx *= -1

        if (self.y <= shift or self.y >= WIND_Y - shift):
            self.vy *= -1
            self.vy -= self.ay

        self.vy += self.ay

    def move(self):
        self._move()

#creating derivative class
class Circle(Figure):
    def __init__(self, x, y, vx, vy, r, ay):
        Figure.__init__(self, x, y, vx, vy)
        self.r = r
        self.ay = ay
    
    def draw(self, img):
#         cv2.circle(img, (int(self.x), int(self.y)), self.r,# + np.random.randint(8),
#                    ((10 + self.x) % 255, (10 + self.y) % 255, 20), -1)
        cv2.circle(img, (int(self.x), int(self.y)), self.r,# + np.random.randint(8),
                       ((201) % 255, (110) % 255, 50), -1)

    def move(self):
        self._move(self.r)

class Line(Figure):
    def __init__(self, x1, y1, vx1, vy1, ay1, x2, y2, vx2, vy2, ay2):
        Figure.__init__(self, 0, 0, 0, 0)
        
        self.p1 = Figure(x1, y1, vx1, vy1, ay1)
        self.p2 = Figure(x2, y2, vx2, vy2, ay2)
    
    def draw(self, img):
        cv2.line(img, (int(self.p1.x), int(self.p1.y)), (int(self.p2.x), int(self.p2.y)),
            ((10 + self.p2.y * 2) % 255, (10 + self.p1.x) % 255, 250), 1)
    
    def move(self):
        self.p1.move()
        self.p2.move()

class Manager:
    def __init__(self, background_color = 50):
        self.objects = []
        
        self.background_color = background_color
        self.canvas = np.ones((WIND_Y, WIND_X, 3), np.uint8) * self.background_color
        self.plot = np.ones((WIND_Y, WIND_X, 3), np.uint8) * self.background_color
        
        self.Ek   = 0
        self.Ep   = 0
        self.exit_needed = False
        
        self.tick_num = 0
            
    def add_wave (self, obj_num, timeout):
        if (self.tick_num == timeout):
            for i in range (obj_num):
                self.add_random_object()
    
    def move(self):
        #move objects
        for obj in self.objects:
            obj.move()
        
        self.Ek = 0
        self.Ep = 0
    
        for obj in self.objects:
            self.Ek += obj.vx**2 + obj.vy**2
            self.Ep += (WIND_Y - obj.y) * obj.ay * 2
    
    def draw(self):
        self.canvas = np.ones((WIND_Y, WIND_X, 3), np.uint8) * self.background_color
        
        #draw objects
        for obj in self.objects:
            obj.draw(self.canvas)
        
        font = cv2.FONT_HERSHEY_SIMPLEX 
        org = (50, 150) 
        fontScale = 2
        color = (255, 0, 0) 
        thickness = 3

        #image = cv2.putText(self.canvas, 'Ek: ' + str(self.Ek), org, font,
        #            fontScale, color, thickness, cv2.LINE_AA) 

        cv2.circle(self.plot, (self.tick_num, WIND_Y - int(self.Ek / 1000)), 3, (10, 21, 200), -1)
        cv2.circle(self.plot, (self.tick_num, WIND_Y - int(self.Ep / 1000)), 3, (198, 28, 20), -1)
        #cv2.circle(self.plot, (self.tick_num, WIND_Y - int(self.Ek) - int(self.Ep)),
        #           3, (198, 228, 220), -1)
        
        self.tick_num += 1
        
        if (self.tick_num >= WIND_X):
            self.tick_num = 0
            self.plot = np.ones((WIND_Y, WIND_X, 3), np.uint8) * self.background_color
        
        #visualise canvas
        cv2.imshow("frame", np.concatenate ((self.canvas, self.plot), axis=1))
        #cv2.imshow("frame", self.canvas)
    
    def add_object(self, new_object):
        self.objects.append(new_object)
    
    def exit(self):
        return self.exit_needed

    def add_random_object(self):
        self.add_object(Circle(np.random.randint(30),
                                   np.random.randint(WIND_Y - 100) + 50,
                                   8 + np.random.random_sample() * 2,
                                   1 + np.random.random_sample() * 3, 6, 0.1))
    
    def handle_keyboard(self):
        key = cv2.waitKey(10) & 0xFF
    
        if (key == ord('q')):
            self.exit_needed = True

        if (key == ord('c')):
            self.add_random_object()

        #if (key == ord('l')):
        #    objects.append(Line(100, 200, 2 + np.random.randint(8), 2 + np.random.randint(7), -0.5,
        #                        100, 200, 2 + np.random.randint(8), 2 + np.random.randint(7), 0.7))
    
    def handle_collisions(self):
        for i in range(len(self.objects)):
            for j in range(i + 1, len(self.objects)):
                obj1 = self.objects[i]
                obj2 = self.objects[j]
                
                if ((obj1.x - obj2.x)**2 + (obj1.y - obj2.y)**2 <= (obj1.r + obj2.r)**2):
                    #print("collision")
                    obj1.vx *= -1
                    obj1.vy *= -1
                    obj2.vx *= -1
                    obj2.vy *= -1
    
manager = Manager()

#main loop
while(manager.exit() == False):
    manager.add_wave(1350, 150)
    manager.move()
    #manager.handle_collisions()
    manager.draw()
    manager.handle_keyboard()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
Ek = m * v^2 / 2
Ep = mgh